In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
import os
import random

class FantasyFootballDraftAssistant:
    def __init__(self):
        self.df = None
        self.positions = {'QB': (1, 2), 'RB': (6, 9), 'WR': (5, 9), 'TE': (1, 2), 'K': (1, 1), 'DST': (1, 1)}
        self.starting_positions = {'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
        self.flex_positions = ['RB', 'WR', 'TE']
        self.q_table = {}
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 0.1  # Exploration rate
        self.total_episodes = 0
        self.base_exploration_N = 24  # Base number of top ADP players to consider during exploration
        self.position_exploration_factor = {
            'QB': 0.5, 'RB': 1.2, 'WR': 1.2, 'TE': 0.8, 'K': 0.3, 'DST': 0.3
        }
        self.epsilon_start = 0.2  # Starting exploration rate
        self.epsilon_end = 0.01  # Ending exploration rate
        self.epsilon_decay = 0.9999  # Decay rate for epsilon
        self.epsilon = self.epsilon_start

    def get_exploration_N(self, round_num, position):
        # Adjust exploration_N based on round number and position
        round_factor = min(2, 1 + (round_num - 1) * 0.1)  # Increase by 10% each round, up to 2x
        position_factor = self.position_exploration_factor.get(position, 1.0)
        return int(self.base_exploration_N * round_factor * position_factor)

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)  
        
    def load_data(self, file_path):
        self.df = pd.read_csv(file_path)
        self.df['ADP'] = pd.to_numeric(self.df['ADP'], errors='coerce')
        self.df = self.df.sort_values('ADP').reset_index(drop=True)
        self.players = self.df.to_dict('records')
        
    def get_state(self, team, round_num):
        pos_counts = [sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()]
        return tuple(pos_counts + [round_num])
    
    def get_actions(self, available_players, team, round_num):
        actions = []
        for player in available_players:
            pos = player['pos']
            pos_count = sum(1 for p in team if p['pos'] == pos)
            if pos_count < self.positions[pos][1] and (pos not in ['K', 'DST'] or round_num > 12):
                actions.append(player['player'])
        return actions
    
    def calculate_team_value(self, team):
        weekly_scores = []
        for week in range(1, 18):  # 17-week season
            available_players = [p for p in team if p['bye_week'] != week]
            starters = self.select_starters(available_players)
            week_score = sum(p['ppr_projection'] / 17 for p in starters)
            weekly_scores.append(week_score)
        
        total_score = sum(weekly_scores)
        bench_strength = sum(p['ppr_projection'] for p in team) - sum(p['ppr_projection'] for p in self.select_starters(team))
        return total_score + (bench_strength * 0.1)  # Adding some value for bench strength
    
    def select_starters(self, available_players):
        starters = []
        for pos, count in self.starting_positions.items():
            if pos == 'FLEX':
                flex_options = sorted([p for p in available_players if p['pos'] in self.flex_positions and p not in starters], 
                                      key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(flex_options[:count])
            else:
                pos_players = sorted([p for p in available_players if p['pos'] == pos and p not in starters], 
                                     key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(pos_players[:count])
        return starters
    
    def calculate_reward(self, team, player, round_num):
        team_value_before = self.calculate_team_value(team)
        new_team = team + [player]
        team_value_after = self.calculate_team_value(new_team)
        value_added = team_value_after - team_value_before
        
        adp_bonus = max(0, (200 - player['ADP']) / 10) if not np.isnan(player['ADP']) else 0
        
        pos_counts = {pos: sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()}
        
        # Prioritize key positions in early rounds
        if round_num <= 6:
            if player['pos'] == 'QB':
                if pos_counts['QB'] == 0:
                    value_added *= 1.5
                else:
                    value_added *= 0.5  # Penalize drafting a second QB early
            elif player['pos'] in ['RB', 'WR']:
                value_added *= 1.5
            elif player['pos'] == 'TE':
                if pos_counts['TE'] == 0:
                    value_added *= 1.5
                else:
                    value_added *= 0.5  # Penalize drafting a second TE early
        
        # Penalize drafting K and DST early
        if player['pos'] in ['K', 'DST'] and round_num <= 12:
            value_added *= 0.1
        
        # Boost value for filling starting lineup
        if pos_counts[player['pos']] < self.starting_positions.get(player['pos'], 0):
            value_added *= 1.5
        
        return value_added + adp_bonus
    
    def train(self, num_episodes=10000, num_teams=12):
        start_episode = self.total_episodes
        for episode in range(start_episode, start_episode + num_episodes):
            if episode % 1000 == 0:
                print(f"Training episode {episode}/{start_episode + num_episodes}")
            
            teams = [[] for _ in range(num_teams)]
            available_players = self.players.copy()
            
            for round_num in range(1, 19):
                for team_index in range(num_teams):
                    state = self.get_state(teams[team_index], round_num)
                    actions = self.get_actions(available_players, teams[team_index], round_num)
                    
                    if not actions:
                        continue
                    
                    if np.random.uniform(0, 1) < self.epsilon:
                        # Exploration: Choose randomly from top N ADP players
                        sorted_actions = sorted(actions, key=lambda a: next((p['ADP'] for p in available_players if p['player'] == a), float('inf')))
                        player = next(p for p in available_players if p['player'] == sorted_actions[0])
                        position = player['pos']
                        exploration_N = self.get_exploration_N(round_num, position)
                        top_N_adp = sorted_actions[:exploration_N]
                        action = np.random.choice(top_N_adp)
                    else:
                        # Exploitation: Choose the action with the highest Q-value
                        q_values = [self.q_table.get((state, a), 0) for a in actions]
                        action = actions[np.argmax(q_values)]
                    
                    player = next(p for p in available_players if p['player'] == action)
                    reward = self.calculate_reward(teams[team_index], player, round_num)
                    teams[team_index].append(player)
                    available_players = [p for p in available_players if p['player'] != action]
                    
                    next_state = self.get_state(teams[team_index], round_num + 1)
                    
                    # Q-value update
                    max_future_q = max([self.q_table.get((next_state, a), 0) for a in self.get_actions(available_players, teams[team_index], round_num + 1)], default=0)
                    current_q = self.q_table.get((state, action), 0)
                    new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_future_q)
                    self.q_table[(state, action)] = new_q
            
            # Decay epsilon after each episode
            self.decay_epsilon()
        
        self.total_episodes += num_episodes
    
    def recommend_players(self, team, available_players, round_num, num_recommendations=5):
        state = self.get_state(team, round_num)
        actions = self.get_actions(available_players, team, round_num)
        q_values = [(a, self.q_table.get((state, a), 0)) for a in actions]
        top_actions = sorted(q_values, key=lambda x: x[1], reverse=True)[:num_recommendations]
        player_dict = {p['player']: p for p in available_players}
        return [(player_dict[a], q) for a, q in top_actions]
    
    def simulate_draft(self, user_position, num_teams=12):
        teams = [[] for _ in range(num_teams)]
        available_players = self.players.copy()
        user_picks = []
        
        for round_num in range(1, 19):
            draft_order = range(num_teams) if round_num % 2 == 1 else reversed(range(num_teams))
            for pick_in_round, team_index in enumerate(draft_order):
                if team_index == user_position:
                    recommendations = self.recommend_players(teams[team_index], available_players, round_num)
                    user_picks.append(((round_num - 1) * num_teams + pick_in_round + 1, recommendations))
                    
                    if recommendations:
                        selected_player, _ = recommendations[0]
                        teams[team_index].append(selected_player)
                        available_players = [p for p in available_players if p['player'] != selected_player['player']]
                else:
                    # Other teams draft based on ADP with round-based randomness
                    valid_players = [p for p in available_players if sum(1 for player in teams[team_index] if player['pos'] == p['pos']) < self.positions[p['pos']][1]]
                    if valid_players:
                        # Sort valid players by ADP, handling NaN values
                        sorted_players = sorted(valid_players, key=lambda p: p['ADP'] if not np.isnan(p['ADP']) else float('inf'))
                        
                        # Determine the number of top players to consider based on the round
                        if round_num <= 1:
                            top_n = 2  # Pick the top ADP in rounds 1 and 2
                        elif round_num <= 3:
                            top_n = 2  # Pick from top 2 ADP in round 3
                        elif 4 <= round_num <= 6:
                            top_n = 3  # Pick from top 3 ADP in rounds 4, 5, 6
                        else:
                            top_n = 5  # Pick from top 5 ADP in later rounds
                        
                        # Select top N players (or fewer if less than N are available)
                        top_players = sorted_players[:min(top_n, len(sorted_players))]
                        
                        # Randomly select one player from the top players
                        player = random.choice(top_players)
                        teams[team_index].append(player)
                        available_players = [p for p in available_players if p['player'] != player['player']]
        
        return user_picks, teams
    
    def save_model(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump((self.q_table, self.total_episodes), f)
        print(f"Model saved to {file_path}")
    
    def load_model(self, file_path):
        with open(file_path, 'rb') as f:
            self.q_table, self.total_episodes = pickle.load(f)
        print(f"Model loaded from {file_path}. Total episodes: {self.total_episodes}")

In [2]:
# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('data//cbs_fantasy_projection_master.csv')

model_file = 'models//fantasy_football_modelV1.pkl'

# Check if a saved model exists
if os.path.exists(model_file):
    assistant.load_model(model_file)
    print("Continuing training from saved model...")
else:
    print("Starting new training...")

# Train for 10,000 episodes
print("Training the model...")
assistant.train(num_episodes=10000)

# Save the model after training
assistant.save_model(model_file)


Starting new training...
Training the model...
Training episode 0/10000
Training episode 1000/10000
Training episode 2000/10000
Training episode 3000/10000
Training episode 4000/10000
Training episode 5000/10000
Training episode 6000/10000
Training episode 7000/10000
Training episode 8000/10000
Training episode 9000/10000
Model saved to models//fantasy_football_modelV1.pkl


In [3]:


user_position=7
print(f"\nSimulating a draft for a user in position {user_position + 1}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 8...

Recommendations for your picks:

Pick 8:
  1. Saquon Barkley (RB) - Q-value: 341.37, ADP: 10.40
  2. Jonathan Taylor (RB) - Q-value: 334.65, ADP: 11.20
  3. Jahmyr Gibbs (RB) - Q-value: 332.60, ADP: 11.60
  4. Kyren Williams (RB) - Q-value: 332.55, ADP: 16.00
  5. Amon-Ra St. Brown (WR) - Q-value: 331.25, ADP: 6.80

Pick 17:
  1. Davante Adams (WR) - Q-value: 298.90, ADP: 18.40
  2. Chris Olave (WR) - Q-value: 296.07, ADP: 23.40
  3. Drake London (WR) - Q-value: 295.21, ADP: 25.20
  4. Nico Collins (WR) - Q-value: 294.98, ADP: 32.80
  5. Deebo Samuel (WR) - Q-value: 294.91, ADP: 33.00

Pick 32:
  1. Nico Collins (WR) - Q-value: 274.89, ADP: 32.80
  2. Deebo Samuel (WR) - Q-value: 274.47, ADP: 33.00
  3. Cooper Kupp (WR) - Q-value: 273.99, ADP: 38.20
  4. DeVonta Smith (WR) - Q-value: 273.45, ADP: 43.40
  5. Brandon Aiyuk (WR) - Q-value: 273.44, ADP: 32.80

Pick 41:
  1. Alvin Kamara (RB) - Q-value: 243.30, ADP: 39.60
  2. Joe Mixon (RB) 

In [54]:


user_position=0
print(f"\nSimulating a draft for a user in position {user_position}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 0...

Recommendations for your picks:

Pick 1:
  1. CeeDee Lamb (WR) - Q-value: 257.48, ADP: 2.00
  2. Tyreek Hill (WR) - Q-value: 255.71, ADP: 3.20
  3. Amon-Ra St. Brown (WR) - Q-value: 254.56, ADP: 6.80
  4. Justin Jefferson (WR) - Q-value: 254.08, ADP: 6.00
  5. Ja'Marr Chase (WR) - Q-value: 252.79, ADP: 5.80

Pick 24:
  1. Drake London (WR) - Q-value: 235.08, ADP: 25.20
  2. Nico Collins (WR) - Q-value: 234.90, ADP: 32.80
  3. Deebo Samuel (WR) - Q-value: 234.61, ADP: 33.00
  4. Mike Evans (WR) - Q-value: 234.34, ADP: 30.80
  5. Cooper Kupp (WR) - Q-value: 234.11, ADP: 38.20

Pick 25:
  1. Jalen Hurts (QB) - Q-value: 223.78, ADP: 31.80
  2. Josh Allen (QB) - Q-value: 223.31, ADP: 26.20
  3. Patrick Mahomes (QB) - Q-value: 221.06, ADP: 29.60
  4. Lamar Jackson (QB) - Q-value: 219.69, ADP: 37.60
  5. C.J. Stroud (QB) - Q-value: 217.10, ADP: 47.60

Pick 48:
  1. George Kittle (TE) - Q-value: 196.57, ADP: 59.00
  2. David Njoku (TE) - Q-value